In [ ]:
import illustris_python as il
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [ ]:
fof_id = np.arange(0,680,1,dtype=int)
job_preamble = 'module reset; \
module restore TNG_h5py; \
export OMP_NUM_THREADS=1; '
job_file_run = 'mpirun -n 16 python ../make_gas_sphere_cutout_fofid.py '

job_string = ''
grouping = 3
counter = 0
for id_num in fof_id:
    if counter == 0:
        job_string += job_preamble
    if counter < grouping - 1:
        job_string += job_file_run + str(id_num) + '; '
        counter += 1
    else:
        job_string += job_file_run + str(id_num) + '\n'
        counter = 0

with open("./jobs/MWcutoutfof_joblist.txt", "w") as file:
    file.write(job_string)
    file.close()

In [ ]:
#run with:
#dsq --job-file MWcutoutfof_joblist.txt --time 1:00:00 -N 1 -n 16 --cpus-per-task=1 --mem=32G --partition=day

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])
job_preamble = 'module reset; \
module restore TNG_h5py; \
export OMP_NUM_THREADS=1; '
job_file_run = 'mpirun -n 16 python ../make_gas_sphere_cutout_subid.py '

job_string = ''
grouping = 3
counter = 0
for id_num in mw_id:
    if counter == 0:
        job_string += job_preamble
    if counter < grouping - 1:
        job_string += job_file_run + str(id_num) + '; '
        counter += 1
    else:
        job_string += job_file_run + str(id_num) + '\n'
        counter = 0

with open("./jobs/MWcutout_joblist.txt", "w") as file:
    file.write(job_string)
    file.close()

In [ ]:
#run with:
#dsq --job-file MWcutout_joblist.txt --time 1:00:00 -N 1 -n 16 --cpus-per-task=1 --mem=32G --partition=day

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])
#fof_id = np.arange(0,680,1,dtype=int)  
job_preamble = 'module reset; \
module restore TNG_h5py; \
export OMP_NUM_THREADS=1; '
job_file_run = 'mpirun -n 16 python ../make_emission_emitvelocity_maps.py '

job_string = ''
n_rotations = 10
#for id_num in mw_id:
for id_num in mw_id:
    job_string += job_preamble
    
    for nrot in range(n_rotations):
        job_string += job_file_run + str(id_num) + ' ' + str(nrot) + '; '
    job_string += '\n'
        
with open("./jobs/Halphaemvelmaps_cold_joblist.txt", "w") as file:
    file.write(job_string)
    file.close()

In [ ]:
#run with:
#dsq --job-file Halphaemvelmaps_cold_joblist.txt --time 8:00:00 -N 1 -n 16 --cpus-per-task=1 --mem=32G --partition=day

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])
#fof_id = np.arange(0,680,1,dtype=int)  
job_preamble = 'module reset; \
module restore TNG_h5py; \
export OMP_NUM_THREADS=1; '
job_file_run = 'mpirun -n 16 python ../make_emission_coldvelocity_maps.py '

job_string = ''
n_rotations = 10
#for id_num in mw_id:
for id_num in mw_id:
    job_string += job_preamble
    
    for nrot in range(n_rotations):
        job_string += job_file_run + str(id_num) + ' ' + str(nrot) + '; '
    job_string += '\n'
        
with open("./jobs/emvelmaps_cold_joblist.txt", "w") as file:
    file.write(job_string)
    file.close()

In [ ]:
#run with:
#dsq --job-file emvelmaps_cold_joblist.txt --time 5:00:00 -N 1 -n 16 --cpus-per-task=1 --mem=32G --partition=day

In [ ]:
from scipy.spatial.transform import Rotation as R

z_hat = np.array([0,0,1])
y_hat = np.array([0,1,0])

w_hat_list = []
v_hat_list = []

theta_list = [['x',0,'y',0],
              ['x',0,'y',30],
              ['x',0,'y',60],
              ['x',0,'y',90],
              ['x',30,'y',0],
              ['x',60,'y',0],
              ['x',90,'y',0],
              ['x',90,'z',30],
              ['x',90,'z',60],
              ['x',45,'z',45]]

for theta in theta_list:
    r1 = R.from_euler(theta[0], theta[1],degrees=True)
    r2 = R.from_euler(theta[2], theta[3],degrees=True)
    w_hat_list.append(r2.apply(r1.apply(z_hat)))
    v_hat_list.append(r2.apply(r1.apply(y_hat)))
w_hat = np.stack(w_hat_list)
v_hat = np.stack(v_hat_list)



In [ ]:
np.dot(w_hat[9],v_hat[9])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(w_hat[:,0],w_hat[:,1],w_hat[:,2],marker='o',s=10)
